<a href="https://colab.research.google.com/github/jocelynbaduria/Deploying-ML-Models-At-Scale/blob/master/Joelyn_Baduria_HW1_part_b_258.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameters and Dataset


In [1]:
BATCH_SIZE =128
EPOCHS =10

training_images_file = 'gs://mnist-public/train-images-idx3-ubyte'
training_labels_file = 'gs://mnist-public/train-labels-idx1-ubyte'
validation_images_file = 'gs://mnist-public/t10k-images-idx3-ubyte'
validation_labels_file = 'gs://mnist-public/t10k-labels-idx1-ubyte'

# Imports Libraries

In [2]:
import os, re, math, json, shutil, pprint
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import IPython.display as display
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt 
print("Tensorflow version" + tf.__version__)

Tensorflow version2.4.1


In [3]:
# @title visualization utilities [RUN ME]
"""
# This cell contains a helper functions used for visualization
and downloads only. You can skip reading it. There is very little useful
Keras/Tensorflow code here.

"""
# Matplotlib config
plt.rc('image', cmap='gray_r')
plt.rc('grid', linewidth=0)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0', figsize=(16,9)) 
# Matplotlib fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")

# Training Reference : https://github.com/GoogleCloudPlatform/tensorflow-without-a-phd/blob/master/tensorflow-mnist-tutorial/keras_01_mnist.ipynb
# pull a batch from the dataset. Reference: 

def dataset_to_numpy_util(training_dataset, validation_dataset, N): 
  # get one batch from each: 10000 validation digits, N training digits
  batch_train_ds = training_dataset.apply(tf.data.experimental.unbatch()).batch(N)

  # eager execution: loop throgh datasets normally
  if tf.executing_eagerly():
    for validation_digits, validation_labels in validation_dataset:
      validation_digits = validation_digits.numpy()
      validation_labels = validation_labels.numpy()
      break
    for training_digits, training_labels in training_dataset:
      training_digits = training_digits.numpy()
      training_labels = training_labels.numpy()
      break

  else:
    v_images, v_labels = validation_dataset.make_one_shot_iterator().get_next()
    t_images, t_labels = batch_train_ds.make_one_shot_iterator().get_next()
    # Run once, get one batch. Session.run returns numpy results
    with tf.Session() as ses:
      (validation_digits, validation_labels, training_digits, training_labels) = ses.run([v_images, v_labels, t_images, t_labels])

  # these were one-hot encoded in the dataset
  validation_labels = np.argmax(validation_labels, axis=1)
  training_labels = np.argmax(training_labels, axis=1)

  return (training_digits, training_labels, validation_digits, validation_labels)

# create digits from local fonts for testing
def create_digits_fron_local_fonts(n):
  font_labels = []
  img = PIL.Image.new('LA', (28*n, 28), color = (0,255)) # format 'LA': black in channel 0, alpha in channel 1
  font1 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'DejaVuSansMono-Oblique.ttf'), 25)
  font2 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'STIXGeneral.ttf'), 25)
  d = PIL.ImageDraw.Draw(img)
  for i in range(n):
    font_labels.append(i%10)
    d.text((7+i*28,0 if i<10 else -4), str(i%10), fill=(255,255), font=font1 if i<10 else font2)
  font_digits = np.array(img.getdata(), np.float32) [:,0] / 255.0 # black in channel 0, alpha in channel 1 (discarded)
  font_digits = np.reshape(np.stack(np.split(np.reshape(font_digits, [28, 28*n]), n, axis=1), axis=0), [n, 28*28])
  return font_digits, font_labels

# utility to display a row of digits with their predictions
def display_digits(digits, predictions, labels, title, n):
  plt.figure(figsize=(13,3))
  digits = np.reshape(digits, [n, 28, 28])
  digits = np.swapaxes(digits, 0, 1)
  digits = np.reshape(digits, [28, 28*n])
  plt.yticks([])
  plt.xticks([28*x+14 for x in range(n)], predictions)
  for i,t in enumerate(plt.gca().xaxis.get_ticklabels()):
    if predictions[i] != labels[i]: t.set_color('red)') # bad predictions in red
  plt.imshow(digits)
  plt.grid(None)
  plt.title(title)
  display.display(fig)

#  utility to display multiple rows of digits, sorted by unrecognized/recognized status
def display_top_unrecognized(digits, predictions, labels, n, lines):
  idx = np.argsort(predicions==labels) # sort order: unrecognized first
  for i in range(lines):
    display_digits(digits[idx][i*n:(i+1)*n], predictions[idx][i*n:(i+1)*n], labels[idx][i*n:(i+1)*n],
                   "{} sample vaildation digits out of {} with bad predictions in red and sorted first".format(n*lines, len(digits)) if i==0 else "", n)

def plot_learning_rate(lr_func, epochs):
  xx = np.arange(epochs+1, dtype=np.float)
  y = [lr_decay(x) for x in xx]
  fig, ax = plt.subplots(figsize=(9, 6))
  ax.set_xlabel('epochs')
  ax.set_title('Learning Rate\ndecays from {:0.3g} to {:0.3g}'.format(y[0], y[-2]))
  ax.minorticks_on()
  ax.grid(True, which='major', axis='both', linestyle='-', linewidth=1)
  ax.grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
  ax.step(xx,y, linewidth=3, where='post')
  display_display(fig)

class PlotTraining(tf.keras.callbacks.Callback):
  def _init_(self, sample_rate=1, zoom=1):
    self.sample_rate = sample_rate
    self.step = 0
    self.zoom = zoom
    self.steps_per_epoch = 60000//BATCH_SIZE

  def on_train_begin(self, logs={}):
    self.batch_history = {}
    self.batch_step = []
    self.epoch_history = {}
    self.epoch_step = []
    self.fig, self.axes = plt.subplots(1, 2, figsize=(16, 7))
    plt.ioff()

  def on_batch_end(self, batch, logs={}):
    if (batch % self.sample_rate) == 0:
      self.batch_step.append(self.step)
      for k,v in logs.items():
        # do not log "batch" and "size" metrics that do not change
        # do not log training accuracy "acc"
        if k=='batch' or k=='size': # or K =='acc':
          continue
        self.batch_history.setdefault(K, []).append(v)
    self.step +=1

  def on_epoch_end(self, epoch, logs={}):
    plt.close(self.fig)
    self.axes[0].cla()
    self.axes[1].cla()

    self.axes[0].set_ylim(0, 1.2/self.zoom)
    self.axes[1].set_ylim(1-1/self.zoom/2, 1+0.1/self.zoom/2)

    self. epoch_step.append(self.step)
    for k,v in logs.items():
      # only log validation metrics
      if not k.startswith('val_'):
        continue
      self.epoch_history.setdefault(k, []).append(v)

      display.clear_output(wait=True)

      for k,v in self.batch_history.items():
        self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.batch_step) / self.steps_per_epoch, v, label=k)

      for k,v in self.epoch_history.items():
        self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.epoch_step) / self.steps_per_epoch, v, label=k, linewidth=3)

      self.axes[0].legend()  
      self.axes[1].legend()
      self.axes[0].set_xlabel('epochs')
      self.axes[1].set_xlabel('epochs')
      self.axes[0].minorticks_on()
      self.axes[0].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
      self.axes[0].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
      self.axes[1].minorticks_on()
      self.axes[1].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
      self.axes[1].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
      display.display(self.fig)  

In [4]:
AUTO = tf.data.experimental.AUTOTUNE

def read_label(tf_bytestring):
  label = tf.io.decode_raw(tf_bytestring, tf.uint8)
  label = tf.reshape(label, [])
  label = tf.one_hot(label, 10)
  return label 

def read_image(tf_bytestring):
  image = tf.io.decode_raw(tf_bytestring, tf.uint8)
  image = tf.cast(image, tf.float32)/256.0
  image = tf.reshape(image, [28*28])
  return image

def load_dataset(image_file, label_file):
  imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes=16)
  imagedataset = imagedataset.map(read_image, num_parallel_calls=16)
  labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8)
  labelsdataset = labelsdataset.map(read_label, num_parallel_calls=16)
  dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
  return dataset

def get_training_dataset(image_file, label_file, batch_size):
  dataset = load_dataset(image_file, label_file)
  dataset = dataset.cache() # this small dataset can be entirely cached in RAM, for TPU this is important to get good performance
  # from such a small dataset
  dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
  dataset = dataset.repeat() # Mandatory for Keras for now
  dataset = dataset.batch(batch_size, drop_remainder=True) # drop_remainder is important on TPU, batch size must be fixed
  dataset = dataset.prefetch(AUTO) # fetch next batches while training on the current (-1: autotune prefetch buffer size)
  return dataset

def get_validation_dataset(image_file, label_file):
  dataset = load_dataset(image_file, label_file)
  dataset = dataset.cache() # this small dataset can be entirely cached in RAM, for TPU this is important to get good performance
  # from such a small dataset
  dataset = dataset.batch(10000, drop_remainder=True) # 10000 items in eval dataset, all in one batch
  dataset = dataset.repeat() # Mandatory for Keras for now
  return dataset

# instantiate the datasets
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_file)

# For TPU, we will need a function that returns the dataset
training_input_fn = lambda: get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_input_fn = lambda: get_validation_dataset(validation_images_file, validation_labels_file)

# Lets have a look at the data


In [5]:
N = 24
(training_digits, training_labels, 
 validation_digits, validation_labels) = dataset_to_numpy_util(training_dataset, validation_dataset, N)
display_digits(training_digits, training_labels, training_labels, "training digits and their labels", N)
display_digits(validation_digits[:N], validation_labels[:N], validation_labels[:N], "validation digits and their labels", N)
font_digits, font_labels = create_digits_from_local_fonts(N)

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.


ValueError: ignored

<Figure size 936x216 with 0 Axes>

# Keras model

In [ ]:
model = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(28*28,)),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

# utility callback that displays training curves
plot_training = PlotTraining(sample_rate=10, zoom=1)